# ACRO Demonstration

In [1]:
import os

import numpy as np
import pandas as pd

In [2]:
# uncomment this line if acro is not installed
# ie you are in development mode
# sys.path.insert(0, os.path.abspath(".."))

In [3]:
from acro import ACRO

### Instantiate ACRO

In [4]:
acro = ACRO(suppress=False)

INFO:acro:version: 0.4.8
INFO:acro:config: {'safe_threshold': 10, 'safe_dof_threshold': 10, 'safe_nk_n': 2, 'safe_nk_k': 0.9, 'safe_pratio_p': 0.1, 'check_missing_values': False, 'survival_safe_threshold': 10, 'zeros_are_disclosive': True}
INFO:acro:automatic suppression: False


### Load test data
The dataset used in this notebook is the nursery dataset from OpenML.  
- In this version, the data can be read directly from the local machine after it has been downloaded. 
- The code below reads the data from a folder called "data" which we assume is at the same level as the folder where you are working.
- The path might need to be changed if the data has been downloaded and stored elsewhere.
 - for example use:  
    path = os.path.join("data", "nursery.arff")  
    if the data is in a sub-folder of your work folder

In [5]:
from scipy.io.arff import loadarff

path = os.path.join("../data", "nursery.arff")
data = loadarff(path)
df = pd.DataFrame(data[0])
df = df.select_dtypes([object])
df = df.stack().str.decode("utf-8").unstack()
df.rename(columns={"class": "recommend"}, inplace=True)
df.head()

,parents,has_nurs,form,children,housing,finance,social,health,recommend
0,usual,proper,complete,1,convenient,convenient,nonprob,recommended,recommend
1,usual,proper,complete,1,convenient,convenient,nonprob,priority,priority
2,usual,proper,complete,1,convenient,convenient,nonprob,not_recom,not_recom
3,usual,proper,complete,1,convenient,convenient,slightly_prob,recommended,recommend
4,usual,proper,complete,1,convenient,convenient,slightly_prob,priority,priority


# Examples of producing tabular output
We rely on the industry-standard package **pandas** for tabulating data.  
In the next few examples we show:
- first, how a researcher would normally make a call in pandas, saving the results in a variable that they can view on screen (or save to file?)
- then how the call is identical in SACRO, except that:
  - "pd" is replaced by "acro"
  - the researcher immediately sees a copy of what the TRE output checker will see.
  

### Pandas crosstab
This is an example of crosstab using pandas.  
We first make the call, then the second line print the outputs to screen.

In [6]:
table = pd.crosstab(df.recommend, df.parents)
print(table)

parents     great_pret  pretentious  usual
recommend                                 
not_recom         1440         1440   1440
priority           858         1484   1924
recommend            0            0      2
spec_prior        2022         1264    758
very_recom           0          132    196


### ACRO crosstab
- This is an example of crosstab using ACRO.  
- The INFO lines show the researcher what will be reported to the output checkers.
- Then the (suppressed as necessary) table is shown via the print command as before.

In [7]:
safe_table = acro.crosstab(
    df.recommend, df.parents, rownames=["recommendation"], colnames=["parents"]
)
print(safe_table)

INFO:acro:get_summary(): fail; threshold: 4 cells may need suppressing; 
INFO:acro:outcome_df:
--------------------------------------------------------|
parents        |great_pret   |pretentious  |usual       |
recommendation |             |             |            |
--------------------------------------------------------|
not_recom      |          ok |          ok |          ok|
priority       |          ok |          ok |          ok|
recommend      | threshold;  | threshold;  | threshold; |
spec_prior     |          ok |          ok |          ok|
very_recom     | threshold;  |          ok |          ok|
--------------------------------------------------------|

INFO:acro:records:add(): output_0


parents         great_pret  pretentious  usual
recommendation                                
not_recom             1440         1440   1440
priority               858         1484   1924
recommend                0            0      2
spec_prior            2022         1264    758
very_recom               0          132    196


### ACRO crosstab with suppression
- This is an example of crosstab with suppressing the cells that violate the disclosure tests.
- Note that you need to change the value of the suppress variable in the acro object to True. Then run the crosstab command.  
- If you wish to continue the research while suppressing the outputs, leave the suppress variable as it is, otherwise turn it off.

In [8]:
acro.suppress = True

safe_table = acro.crosstab(df.recommend, df.parents)
print(safe_table)

INFO:acro:get_summary(): fail; threshold: 4 cells suppressed; 
INFO:acro:outcome_df:
----------------------------------------------------|
parents    |great_pret   |pretentious  |usual       |
recommend  |             |             |            |
----------------------------------------------------|
not_recom  |          ok |          ok |          ok|
priority   |          ok |          ok |          ok|
recommend  | threshold;  | threshold;  | threshold; |
spec_prior |          ok |          ok |          ok|
very_recom | threshold;  |          ok |          ok|
----------------------------------------------------|

INFO:acro:records:add(): output_1


parents     great_pret  pretentious   usual
recommend                                  
not_recom       1440.0       1440.0  1440.0
priority         858.0       1484.0  1924.0
recommend          NaN          NaN     NaN
spec_prior      2022.0       1264.0   758.0
very_recom         NaN        132.0   196.0


## An example of a  more complex table
- make the children variable numeric
- so we can report statistics like mean etc.

In [9]:
df["children"].replace(to_replace={"more": "4"}, inplace=True)
df["children"] = pd.to_numeric(df["children"])

df["children"] = df.apply(
    lambda row: (
        row["children"] if row["children"] in (1, 2, 3) else np.random.randint(4, 10)
    ),
    axis=1,
)

In [10]:
acro.suppress = False
acro.crosstab(
    index=[df.parents, df.finance],
    columns=[df.recommend],
    values=df.children,
    aggfunc="mean",
    margins="total",
)

INFO:acro:get_summary(): fail; threshold: 2 cells may need suppressing; p-ratio: 9 cells may need suppressing; nk-rule: 9 cells may need suppressing; 
INFO:acro:outcome_df:
-----------------------------------------------------------------------------------------------------------------|
|recommend             | not_recom| priority recommend                      |spec_prior |very_recom          |All|
|parents     finance   |          |                                         |           |                    |   |
-----------------------------------------------------------------------------------------------------------------|
|great_pret  convenient|  ok      |  ok                  p-ratio; nk-rule;  | ok        | p-ratio; nk-rule;  | ok|
|            inconv    |  ok      |  ok                  p-ratio; nk-rule;  | ok        | p-ratio; nk-rule;  | ok|
|pretentious convenient|  ok      |  ok                  p-ratio; nk-rule;  | ok        |                 ok | ok|
|            inconv   

recommend               not_recom  priority  recommend  spec_prior  \
parents     finance                                                  
great_pret  convenient   3.138889  2.787109        NaN    3.324353   
            inconv       3.131944  2.393064        NaN    3.411335   
pretentious convenient   3.087500  3.104167        NaN    3.270548   
            inconv       3.090278  3.057263        NaN    3.319118   
usual       convenient   3.127778  3.117220        1.0    3.328488   
            inconv       3.126389  3.163542        NaN    3.371981   
All                      3.117130  3.016878        1.0    3.344461   

recommend               very_recom       All  
parents     finance                           
great_pret  convenient         NaN  3.135185  
            inconv             NaN  3.155093  
pretentious convenient    2.647727  3.125000  
            inconv        1.363636  3.116204  
usual       convenient    2.600000  3.121296  
            inconv        1.363636  3.136111  
All                       2.198171  3.131481

# ACRO functionality to let users manage their outputs

### 1: List current ACRO outputs
This is an example of using the print_output function to list all the outputs created so far

In [11]:
_ = acro.print_outputs()

uid: output_0
status: fail
type: table
properties: {'method': 'crosstab'}
sdc: {'summary': {'suppressed': False, 'negative': 0, 'missing': 0, 'threshold': 4, 'p-ratio': 0, 'nk-rule': 0, 'all-values-are-same': 0}, 'cells': {'negative': [], 'missing': [], 'threshold': [[2, 0], [2, 1], [2, 2], [4, 0]], 'p-ratio': [], 'nk-rule': [], 'all-values-are-same': []}}
command: safe_table = acro.crosstab(
summary: fail; threshold: 4 cells may need suppressing; 
outcome: parents          great_pret  pretentious        usual
recommendation                                       
not_recom                ok           ok           ok
priority                 ok           ok           ok
recommend       threshold;   threshold;   threshold; 
spec_prior               ok           ok           ok
very_recom      threshold;            ok           ok
output: [parents         great_pret  pretentious  usual
recommendation                                
not_recom             1440         1440   1440
priority  

### 2: Remove some ACRO outputs before finalising 
This is an example of deleting some of the ACRO outputs.  
The name of the output that needs to be removed should be passed to the function remove_output.  
- The output name can be taken from the outputs listed by the print_outputs function, 
- or by listing the results and choosing the specific output that needs to be removed

In [12]:
acro.remove_output("output_0")

INFO:acro:records:remove(): output_0 removed


### 3: Rename ACRO outputs before finalising
This is an example of renaming the outputs to provide a more descriptive name.

In [13]:
acro.rename_output("output_1", "cross_tabulation")

INFO:acro:records:rename_output(): output_1 renamed to cross_tabulation


### 4: Add a comment to output
This is an example to add a comment to outputs.  
It can be used to provide a description or to pass additional information to the output checkers.

In [14]:
acro.add_comments(
    "cross_tabulation", "Suppression has been applied. Please let me have this data."
)

INFO:acro:records:a comment was added to cross_tabulation


### 5. Request an exception
An example of providing a reason why an exception should be made

In [15]:
acro.add_exception("output_2", "This is evidence of systematic bias?")

INFO:acro:records:exception request was added to output_2


## 5: (the big one) Finalise ACRO
This is an example of the function _finalise()_ which the users must call at the end of each session.  
- It takes each output and saves it to a CSV file.    
- It also saves the SDC analysis for each output to a json file or Excel file  
  (depending on the extension of the name of the file provided as an input to the function)
- If an output is flagged as potentially disclosive then the  
  researcher is prompted to provide a reason for release if they have not already done so.

In [17]:
output = acro.finalise("Examples", "json")

INFO:acro:records:
uid: cross_tabulation
status: fail
type: table
properties: {'method': 'crosstab'}
sdc: {'summary': {'suppressed': True, 'negative': 0, 'missing': 0, 'threshold': 4, 'p-ratio': 0, 'nk-rule': 0, 'all-values-are-same': 0}, 'cells': {'negative': [], 'missing': [], 'threshold': [[2, 0], [2, 1], [2, 2], [4, 0]], 'p-ratio': [], 'nk-rule': [], 'all-values-are-same': []}}
command: safe_table = acro.crosstab(df.recommend, df.parents)
summary: fail; threshold: 4 cells suppressed; 
outcome: parents      great_pret  pretentious        usual
recommend                                        
not_recom            ok           ok           ok
priority             ok           ok           ok
recommend   threshold;   threshold;   threshold; 
spec_prior           ok           ok           ok
very_recom  threshold;            ok           ok
output: [parents     great_pret  pretentious   usual
recommend                                  
not_recom       1440.0       1440.0  1440.0
priori

 suppressed


INFO:acro:records:outputs written to: Examples
